In [74]:
from concorde.tsp import TSPSolver
import tsplib95

import numpy as np
import json
import re

In [75]:
PARAMETERS_FILE = '../parameters.json'

In [76]:
with open(PARAMETERS_FILE) as f:
    parameters = json.load(f)

pattern = r'/playlist/([^/?]+)'

playlist_id = list()

for url in parameters['playlist_id']:
    match = re.search(pattern, url)

    if match:
        playlist_id.append(match.group(1))

playlist_id

['3tTo8cKpIQbLJ3MTzgEVjV', '1Kzd8jonR1022i8dL8eDda', '3bxNKleDijOT6Did05jINs']

In [77]:
data = dict()

for _id in playlist_id:
    with open(f"../data/metadata/{_id}.json") as f:
        data[_id] = json.load(f)

In [78]:
def compare_artist_album_similarity(data_a:dict, data_b:dict) -> int:
    distance = 0

    if (data_a["artist"] == data_b["artist"]):
        distance += parameters["same_artist_bonus"]
        
        if (data_a["album"] == data_b["album"]):
            distance += parameters["same_album_bonus"]

    return distance


def compare_intersections(a:list, b:list, multiplier:list) -> int:
    if a and b:
        set_a = set(a)
        set_b = set(b)

        intersection = set_a.intersection(set_b)

        intersection_size = len(intersection)

        if intersection_size > 0:
        
            total_size = len(set_a) + len(set_b)

            CONSTANT = 100

            similarity = (intersection_size / total_size) * CONSTANT

            return similarity *multiplier

    return 0

In [79]:
def initialize_adjacency_matrix(size:int) -> list:
    return [[0 for col in range(size)] for row in range(size)]


def populate_adjacency_matrix(adjacency_matrix:list, size:int, data) -> list:
    for i in range(size):
        for j in range(size):
            if i != j:
                distance = parameters['initial_distance']

                distance -= compare_artist_album_similarity(data[i], data[j])

                distance -= compare_intersections(
                    data[i]["genres"], data[j]["genres"], parameters["genres_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["related_genres"], data[j]["related_genres"], parameters["related_genres_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["related_artists"], data[j]["related_artists"], parameters["related_artists_intersection_multiplier"])
                distance -= compare_intersections(
                    data[i]["generic_genres"], data[j]["generic_genres"], parameters["generic_genres_intersection_multiplier"])

                adjacency_matrix[i][j] = int(distance);
    
    return adjacency_matrix


def print_adjacency_matrix(adjacency_matrix:list, size:int) -> None:
    for i in range(size):
        for j in range(size):
            print("{}".format(adjacency_matrix[i][j]), end=" ")
        print()    
        

In [80]:
for _id, _metadata in data.items():
    print(_id, _metadata)

    SIZE = len(_metadata)
    adjacency_matrix = initialize_adjacency_matrix(SIZE)
    adjacency_matrix = populate_adjacency_matrix(adjacency_matrix, SIZE, _metadata)
    # print_adjacency_matrix(adjacency_matrix, SIZE)

    problem = tsplib95.models.StandardProblem()

    problem.name = _id
    problem.type = "TSP"
    problem.dimension = SIZE
    problem.edge_weight_type = "EXPLICIT"
    problem.edge_weight_format = "FULL_MATRIX"
    problem.node_coord_type = "NO_COORDS"
    problem.display_data_type = "NO_DISPLAY"
    problem.edge_weights = np.array(adjacency_matrix).tolist()

    problem.save(f"../data/tsp/{_id}.tsp")


3tTo8cKpIQbLJ3MTzgEVjV [{'id': '4sQdW03IQh3tsZeBELdt5G', 'name': 'Foxey Lady', 'album': 'Are You Experienced', 'artist': 'Jimi Hendrix', 'genres': ['acid rock', 'album rock', 'classic rock', 'hard rock', 'proto-metal', 'psychedelic rock', 'rock'], 'related_artists': ['Cream', 'The Yardbirds', 'Canned Heat', 'Derek & The Dominos', 'Steppenwolf', 'Janis Joplin', 'Ten Years After', 'Free', 'Jefferson Airplane', 'The Doors', 'The Animals', 'Big Brother & The Holding Company', 'Stevie Ray Vaughan', 'Rory Gallagher', 'Jim Morrison', 'Traffic', 'John Mayall & The Bluesbreakers', 'Allman Brothers Band', 'Albert King', 'The Who'], 'related_genres': ['roots rock', 'southern rock', 'blues rock', 'heartland rock', 'texas blues', 'soul blues', 'electric blues', 'jam band', 'blues', 'soft rock', 'singer-songwriter', 'traditional blues', 'mellow gold', 'symphonic rock', 'folk rock', 'british invasion', 'progressive rock', 'british blues', 'supergroup', 'country rock', 'folk', 'protopunk', 'classic ca

In [81]:
for _id, _metadata in data.items():
    solver = TSPSolver.from_tspfile(f"../data/tsp/{_id}.tsp")

    solution = solver.solve()

    solution_dict = dict()

    solution_dict['found_tour'] = solution.found_tour
    solution_dict['success'] = solution.success
    solution_dict['hit_timebound'] = solution.hit_timebound
    solution_dict['tour'] = solution.tour.tolist()
    solution_dict['optimal_value'] = solution.optimal_value

    solution_dict['tour_ids'] = [_metadata[vertex]['id'] for vertex in solution.tour]
    solution_dict['tour_names'] = [_metadata[vertex]['name'] for vertex in solution.tour]

    with open(f"../data/solutions/{_id}.json", "w") as f:
        json.dump(solution_dict, f, indent=4)        


Problem Name: 3tTo8cKpIQbLJ3MTzgEVjV
Problem Type: TSP
Number of Nodes: 77
Explicit Lengths (CC_MATRIXNORM)
CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Setting kick type to close
Starting Cycle: 27484
   0 Steps   Best: 27447   0.01 seconds
  20 Steps   Best: 27367   0.13 seconds
  22 Steps   Best: 27364   0.14 seconds
  30 Steps   Best: 27250   0.18 seconds
  38 Total Steps.
Best cycle length: 27250
Lin-Kernighan Running Time: 0.23
LK Initial Run: 27250.0
LK Run 0: 27250.0
LK Run from best tour: 27250.0
Time to find compression tour: 0.79 (seconds)
Set initial upperbound to 27250 (from tour)
Fractional Matching: 25914.5
Initial Running Time: 0.00 (seconds)
Basis Running Time: 0.00 (seconds)
Total fractional matching time: 0.00 (seconds)
Total Time for first_lp: 0.00 (seconds)
Setting upperbound to the initial bound: 27250.00
Loading lp...done in 0.01 seconds
LP has:  77 rows  165 columns  330 nonzeros
Dual opt returned after 0.00 seconds
Initial LP value: 2

CCtsp_solve_dat ...
Finding a good tour for compression ...
linkern ...
Less than 10 nodes, setting repeatcount to 0
Setting kick type to close
Starting Cycle: 7063
LK Cycle: 7063
Best cycle length: 7063
Lin-Kernighan Running Time: 0.00
LK Initial Run: 7063.0
Less than 10 nodes, setting repeatcount to 0
LK Run 0: 7063.0
Less than 10 nodes, setting repeatcount to 0
LK Run from best tour: 7063.0
Time to find compression tour: 0.01 (seconds)
Set initial upperbound to 7063 (from tour)
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Less than 10 nodes, setting repeatcount to 0
Fractional Matching: 7063.0
Initial Running Time: 0.00 (seconds)